In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

# Programming with Python

## Control Flow Statements

The control flow of  a program determines the order in which lines of code are executed. All else being equal, Python code is executed linearly, in the order that lines appear in the program. However, all is not usually equal, and so the appropriate control flow is frequently specified with the help of control flow statements. These include loops, conditional statements and calls to functions. Let’s look at a few of these here.

### for statements
One way to repeatedly execute a block of statements (*i.e.* loop) is to use a `for` statement. These statements iterate over the number of elements in a specified sequence, according to the following syntax:

In [2]:
for letter in 'ciao':
    print('give me a {0}'.format(letter.upper()))

give me a C
give me a I
give me a A
give me a O


Recall that strings are simply regarded as sequences of characters. Hence, the above `for` statement loops over each letter, converting each to upper case with the `upper()` method and printing it. 

Similarly, as shown in the introduction, **list comprehensions** may be constructed using `for` statements:

In [3]:
[i**2 for i in range(10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

Here, the expression loops over `range(10)` -- the sequence from 0 to 9 -- and squares each before placing it in the returned list.

### if statements
As the name implies, `if` statements execute particular sections of code depending on some tested **condition**. For example, to code an absolute value function, one might employ conditional statements:

In [4]:
def absval(some_list):

    # Create empty list
    absolutes = []    

    # Loop over elements in some_list
    for value in some_list:

        # Conditional statement
        if value<0:
            # Negative value
            absolutes.append(-value)

        else:
            # Positive value
            absolutes.append(value)
    
    return absolutes 

Here, each value in `some_list` is tested for the condition that it is negative, in which case it is multiplied by -1, otherwise it is appended as-is. 

For conditions that have more than two possible values, the `elif` clause can be used:

In [5]:
x = 5
if x < 0:
    print('x is negative')
elif x % 2:
    print('x is positive and odd')
else:
    print('x is even and non-negative')

x is positive and odd


### while statements

A different type of conditional loop is provided by the `while` statement. Rather than iterating a specified number of times, according to a given sequence, `while` executes its block of code repeatedly, until its condition is no longer true. 

For example, suppose we want to sample from a truncated normal distribution, where we are only interested in positive-valued samples. The following function is one solution:

In [6]:
# Import function
from numpy.random import normal

def truncated_normals(how_many, l):

    # Create empty list
    values = []

    # Loop until we have specified number of samples
    while (len(values) < how_many):

        # Sample from standard normal
        x = normal(0,1)

        # Append if not truncateed
        if x > l: values.append(x)

    return values    

In [7]:
truncated_normals(15, 0)

[0.052445290112249285,
 0.5829957736256567,
 0.3956524177833495,
 0.6732739868440062,
 0.7741087625055059,
 0.7295687474194539,
 2.1166389235910334,
 0.10198439031488121,
 0.08219910284942006,
 0.33771516033285426,
 1.4264930315802709,
 0.7705230383377356,
 1.222931972188232,
 0.8842790476249497,
 1.1970014893281016]

This function iteratively samples from a standard normal distribution, and appends it to the output array if it is positive, stopping to return the array once the specified number of values have been added.
	
Obviously, the body of the `while` statement should contain code that eventually renders the condition false, otherwise the loop will never end! An exception to this is if the body of the statement contains a `break` or `return` statement; in either case, the loop will be interrupted.


## Generators

When a Python functions is called, it creates a namespace for the function, executes the code that comprises the function (creating objects inside the namespace as required), and returns some result to its caller. After the return, everything inside the namespace (including the namespace itself) is gone, and is created anew when the function is called again. 

However, one particular class of functions in Python breaks this pattern, returning a value to the caller while still active, and able to return subsequent values as needed. Python ***generators*** employ `yield` statements in place of `return`, allowing a sequence of values to be generated without having to create a new function namespace each time. In other languages, this construct is known as a *coroutine*. 

For example, we may want to have a function that returns a sequence of values; let's consider, for a simple illustration, the Fibonacci sequence:

$$F_i = F_{i-2} + F_{i-1}$$

its certaintly possible to write a standard Python function that returns however many Fibonacci numbers that we need:

In [8]:
import numpy as np

def fibonacci(size):
    F = np.empty(size, 'int')
    a, b = 0, 1
    for i in range(size):
        F[i] = a
        a, b = b, a + b
    return F

and this works just fine:

In [9]:
fibonacci(20)

array([   0,    1,    1,    2,    3,    5,    8,   13,   21,   34,   55,
         89,  144,  233,  377,  610,  987, 1597, 2584, 4181])

However, what if we need one number at a time, or if we need a million or 10 million values? In the first case, you would somehow have to store the values from the last iteration, and restore the state to the function each time it is called. In the second case, you would have to generate and then store a very large number of values, most of which you may not need right now.

A more sensible solution is to create a `generator`, which calculates a single value in the sequence, then *returns control back to the caller*. This allows the generator to be called again, resuming the sequence generation where it left off. Here's the Fibonacci function, implemented as a generator:

In [10]:
def gfibonacci(size):
    a, b = 0, 1
    for _ in range(size):
        yield a
        a, b = b, a + b

Notice that there is no `return` statement at all; just `yield`, which is where a value is returned each time one is requested. The `yield` statement is what defines a generator. 

When we call our generator, rather than a sequence of Fibonacci numbers, we get a generator object:

In [11]:
f = gfibonacci(100)
f

<generator object gfibonacci at 0x10c16ae58>

A generator has a `__next__()` method that can be called via the builtin function `next()`. The call to `next` executes the generator until the `yield` statement is reached, returning the next generated value, and then pausing until another call to `next` occurs:

In [12]:
next(f), next(f), next(f)

(0, 1, 1)

A generator is a type of `iterator`. If we call a function that supports iterables using a generator as an argument, it will know how to use the generator.

In [13]:
np.array(list(f))

array([2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584,
       4181, 6765, 10946, 17711, 28657, 46368, 75025, 121393, 196418,
       317811, 514229, 832040, 1346269, 2178309, 3524578, 5702887, 9227465,
       14930352, 24157817, 39088169, 63245986, 102334155, 165580141,
       267914296, 433494437, 701408733, 1134903170, 1836311903, 2971215073,
       4807526976, 7778742049, 12586269025, 20365011074, 32951280099,
       53316291173, 86267571272, 139583862445, 225851433717, 365435296162,
       591286729879, 956722026041, 1548008755920, 2504730781961,
       4052739537881, 6557470319842, 10610209857723, 17167680177565,
       27777890035288, 44945570212853, 72723460248141, 117669030460994,
       190392490709135, 308061521170129, 498454011879264, 806515533049393,
       1304969544928657, 2111485077978050, 3416454622906707,
       5527939700884757, 8944394323791464, 14472334024676221,
       23416728348467685, 37889062373143906, 61305790721611591,
       9919485309475

What happens when we reach the "end" of a generator?

In [14]:
a_few_fibs = gfibonacci(2)

In [15]:
next(a_few_fibs)

0

In [16]:
next(a_few_fibs)

1

In [17]:
next(a_few_fibs)

StopIteration: 

Thus, generators signal when there are no further values to generate by throwing a `StopIteration` exception. We must either handle this exception, or create a generator that is infinite, which we can do in this example by replacing a `for` loop with a `while` loop:

In [19]:
def infinite_fib():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

In [20]:
f = infinite_fib()
vals = [next(f) for _ in range(10000)]
vals[-1]

2079360823713349807211264898864283682508703609401590311968294586652850142345568664892745603430522651559175734329719015801062479426725097317613381017990273803823178974834623555648319143159192453239442002806781032040872441469346284906266838708330804825092065449334087873322637758084744632487379760373479464825811385863155040408101726038120291994389237094285260164739821355447908182359371542956694514931299366484677909043779928477367537928427066017513466483326637769864201210689135579114187277693408080350495679409464829288056605636471818766266897075853738335267742083557415594565854200363476532454100612101244678568917149480326240860269309121160197393822944663604990153196328615969907788042772028923553932967187718291564341907918652511867885682160089752017107049943765706734240087108390881180097625972743182053955425686946081535591845825339823438236043576275982317989611674842426954592463320461413799285081435201873848092358155398899089715146940613169561449778372074346137375621868510685682609069633981

## Error Handling

Inevitably, some code you write will generate errors, at least in some situations. Unless we explicitly anticipate and **handle** these errors, they will cause your code to halt (sometimes this is a good thing!). Errors are handled using `try/except` blocks.

If code executed in the `try` block generates an error, code execution moves to the `except` block. If the exception that is specified corresponsd to that which has been raised, the code in the `except` block is executed before continuing; otherwise, the exception is carried out and the code is halted.

In [21]:
absval(-5)

TypeError: 'int' object is not iterable

In the call to `absval`, we passed a single negative integer, whereas the function expects some sort of iterable data structure. Other than changing the function itself, we can avoid this error using exception handling.

In [22]:
x = -5
try:
    print(absval(x))
except TypeError:
    print('The argument to absval must be iterable!')

The argument to absval must be iterable!


In [23]:
x = -5
try:
    print(absval(x))
except TypeError:
    print(absval([x]))

[5]


We can raise exceptions manually by using the `raise` expression.

In [24]:
raise ValueError('This is the wrong value')

ValueError: This is the wrong value

## Importing and Manipulating Data

Python includes operations for importing and exporting data from files and binary objects, and third-party packages exist for database connectivity. The easiest way to import data from a file is to parse **delimited** text file, which can usually be exported from spreadsheets and databases. In fact, file is a built-in type in python. Data may be read from and written to regular files by specifying them as file objects:

In [25]:
microbiome = open('../data/microbiome.csv')

Here, a file containing microbiome data in a comma-delimited format is opened, and assigned to an object, called `microbiome`. The next step is to transfer the information in the file to a usable data structure in Python. Since this dataset contains four variables, the name of the taxon, the patient identifier (de-identified), the bacteria count in tissue and the bacteria count in stool, it is convenient to use a dictionary. This allows each variable to be specified by name. 


First, a dictionary object is initialized, with appropriate keys and corresponding lists, initially empty. Since the file has a header, we can use it to generate an empty dict:

In [26]:
column_names = next(microbiome).rstrip('\n').split(',')
column_names

['Taxon', 'Patient', 'Group', 'Tissue', 'Stool']

> **Compatibility Corner**: In Python 2, `open` would not return a generator, but rather a `file` object with a `next` method. In Python 3, an generator is returned, which requires the use of the built-in function `next`.

In [27]:
mb_dict = {name:[] for name in column_names}

In [28]:
mb_dict

{'Group': [], 'Patient': [], 'Stool': [], 'Taxon': [], 'Tissue': []}

In [29]:
for line in microbiome:
    taxon, patient, group, tissue, stool = line.rstrip('\n').split(',')
    mb_dict['Taxon'].append(taxon)
    mb_dict['Patient'].append(int(patient))
    mb_dict['Group'].append(int(group))
    mb_dict['Tissue'].append(int(tissue))
    mb_dict['Stool'].append(int(stool))

For each line in the file, data elements are split by the comma delimiter, using the `split` method that is built-in to string objects. Each datum is subsequently appended to the appropriate list stored in the dictionary. After all the data is parsed, it is polite to close the file:

In [30]:
microbiome.close()

The data can now be readily accessed by indexing the appropriate variable by name:

In [31]:
mb_dict['Tissue'][:10]

[136, 1174, 408, 831, 693, 718, 173, 228, 162, 372]

A second approach to importing data involves interfacing directly with a relational database management system. Relational databases are far more efficient for storing, maintaining and querying data than plain text files or spreadsheets, particularly for large datasets or multiple tables. A number of third parties have created packages for database access in Python. For example, `sqlite3` is a package that provides connectivity for SQLite databases:

In [32]:
import sqlite3
db = sqlite3.connect(database='../data/baseball-archive-2011.sqlite')

# create a cursor object to communicate with database
cur = db.cursor()   

In [33]:
# run query
cur.execute('SELECT playerid, HR, SB FROM Batting WHERE yearID=1970')

# fetch data, and assign to variable
baseball = cur.fetchall() 
baseball[:10]

[('aaronha01', 38, 9),
 ('aaronto01', 2, 0),
 ('abernte02', 0, 0),
 ('abernte02', 0, 0),
 ('abernte02', 0, 0),
 ('acosted01', 0, 0),
 ('adairje01', 0, 0),
 ('ageeto01', 24, 31),
 ('aguirha01', 0, 0),
 ('akerja01', 0, 0)]

## Functions

Python uses the `def` statement to encapsulate code into a callable function. Here again is a very simple Python function:

In [34]:
# Function for calulating the mean of some data
def mean(data):

    # Initialize sum to zero
    sum_x = 0.0

    # Loop over data
    for x in data:

        # Add to sum
        sum_x += x 
    
    # Divide by number of elements in list, and return
    return sum_x / len(data)

As we can see, arguments are specified in parentheses following the function name. If there are sensible "default" values, they can be specified as a **keyword argument**.

In [35]:
def var(data, sample=True):

    # Get mean of data from function above
    x_bar = mean(data)

    # Do sum of squares in one line
    sum_squares = sum([(x - x_bar)**2 for x in data])

    # Divide by n-1 and return
    if sample:
        return sum_squares/(len(data)-1)
    return sum_squares/len(data)

Non-keyword arguments must always predede keyword arguments, and must always be presented in order; order is not important for keyword arguments.

Arguments can also be passed to functions as a `tuple`/`list`/`dict` using the asterisk notation.

In [36]:
def some_computation(a=-1, b=4.3, c=7):
    return (a + b) / float(c)

args = (5, 4, 3)
some_computation(*args)

3.0

In [37]:
kwargs = {'b':4, 'a':5, 'c':3}
some_computation(**kwargs)

3.0

The `lambda` statement creates anonymous one-line functions that can simply be assigned to a name.

In [38]:
import numpy as np
normalize = lambda data: (np.array(data) - np.mean(data)) / np.std(data)

or not:

In [39]:
(lambda data: (np.array(data) - np.mean(data)) / np.std(data))([5,8,3,8,3,1,2,1])

array([ 0.42192651,  1.54706386, -0.32816506,  1.54706386, -0.32816506,
       -1.07825663, -0.70321085, -1.07825663])

Python has several built-in, higher-order functions that are useful.

In [40]:
list(filter(lambda x: x > 5, range(10)))

[6, 7, 8, 9]

In [41]:
abs([5,-6])

TypeError: bad operand type for abs(): 'list'

In [42]:
list(map(abs, [5, -6]))

[5, 6]

## Example: Least Squares Estimation

Lets try coding a statistical function. Suppose we want to estimate the parameters of a simple linear regression model. The objective of regression analysis is to specify an equation that will predict some response variable $Y$ based on a set of predictor variables $X$. This is done by fitting parameter values $\beta$ of a regression model using extant data for $X$ and $Y$. This equation has the form:

$$Y = X\beta + \epsilon$$

where $\epsilon$ is a vector of errors. One way to fit this model is using the method of *least squares*, which is given by:

$$\hat{\beta} = (X^{\prime} X)^{-1}X^{\prime} Y$$

We can write a function that calculates this estimate, with the help of some functions from other modules:

In [43]:
from numpy.linalg import inv
from numpy import transpose, array, dot

We will call this function `solve`, requiring the predictor and response variables as arguments. For simplicity, we will restrict the function to univariate regression, whereby only a single slope and intercept are estimated:

In [44]:
def solve(x,y):
    'Estimates regession coefficents from data'

    '''
    The first step is to specify the design matrix. For this, 
    we need to create a vector of ones (corresponding to the intercept term, 
    and along with x, create a n x 2 array:
    '''
    X = array([[1]*len(x), x])

    '''
    An array is a data structure from the numpy package, similar to a list, 
    but allowing for multiple dimensions. Next, we calculate the transpose of x, 
    using another numpy function, transpose
    '''
    Xt = transpose(X)

    '''
    Finally, we use the matrix multiplication function dot, also from numpy 
    to calculate the dot product. The inverse function is provided by the LinearAlgebra 
    package. Provided that x is not singular (which would raise an exception), this 
    yields estimates of the intercept and slope, as an array
    '''
    b_hat = dot(inv(dot(X,Xt)), dot(X,y))

    return b_hat 

Here is solve in action:

In [45]:
solve((10,5,10,11,14),(-4,3,0,23,0.6))

array([ 2.04380952,  0.24761905])

## Object-oriented Programming

As previously stated, Python is an object-oriented programming (OOP) language, in contrast to procedural languages like FORTRAN and C. As the name implies, object-oriented languages employ objects to create convenient abstractions of data structures. This allows for more flexible programs, fewer lines of code, and a more natural programming paradigm in general. An object is simply a modular unit of data and associated functions, related to the state and behavior, respectively,  of some abstract entity. Object-oriented languages group similar objects into classes. For example, consider a Python class representing a bird:

In [46]:
class bird:
    # Class representing a bird

    name = 'bird'
    
    def __init__(self, sex):
        # Initialization method
        
        self.sex = sex

    def fly(self):
        # Makes bird fly

        print('Flying!')
        
    def nest(self):
        # Makes bird build nest

        print('Building nest ...')
        
    @classmethod
    def get_name(cls):
        # Class methods are shared among instances
        
        return cls.name

You will notice that this `bird` class is simply a container for two functions (called *methods* in Python), `fly` and `nest`, as well as one attribute, `name`. The methods represent functions in common with all members of this class. You can run this code in Python, and create birds:

In [47]:
Tweety = bird('male')
Tweety.name

'bird'

In [48]:
Tweety.fly()

Flying!


In [49]:
Foghorn = bird('male')
Foghorn.nest()

Building nest ...


A `classmethod` can be called without instantiating an object.

In [50]:
bird.get_name()

'bird'

Whereas standard methods cannot:

In [51]:
bird.fly()

TypeError: fly() missing 1 required positional argument: 'self'

As many instances of the `bird` class can be generated as desired, though it may quickly become boring. One of the important benefits of using object-oriented classes is code re-use. For example, we may want more specific kinds of birds, with unique functionality:

In [52]:
class duck(bird):
    # Duck is a subclass of bird

    name = 'duck'
    
    def swim(self):
        # Ducks can swim

        print('Swimming!')

    def quack(self,n):
        # Ducks can quack
    
        print('Quack! ' * n)

Notice that this new `duck` class refers to the `bird` class in parentheses after the class declaration; this is called **inheritance**. The subclass `duck` automatically inherits all of the variables and methods of the superclass, but allows new functions or variables to be added. In addition to flying and best-building, our duck can also swim and quack:

In [53]:
Daffy = duck('male')
Daffy.swim()

Swimming!


In [54]:
Daffy.quack(3)

Quack! Quack! Quack! 


In [55]:
Daffy.nest()

Building nest ...


Along with adding new variables and methods, a subclass can also override existing variables and methods of the superclass. For example, one might define `fly` in the `duck` subclass to return an entirely different string. It is easy to see how inheritance promotes code re-use, sometimes dramatically reducing development time. Classes which are very similar need not be coded repetitiously, but rather, just extended from a single superclass. 

This brief introduction to object-oriented programming is intended only to introduce new users of Python to this programming paradigm. There are many more salient object-oriented topics, including interfaces, composition, and introspection. I encourage interested readers to refer to any number of current Python and OOP books for a more comprehensive treatment.

## In Python, everything is an object

Everything (and I mean *everything*) in Python is an object, in the sense that they possess attributes, such as methods and variables, that we usually associate with more "structured" objects like those we created above.

Check it out:

In [56]:
dir(1)

['__abs__',
 '__add__',
 '__and__',
 '__bool__',
 '__ceil__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floor__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__index__',
 '__init__',
 '__int__',
 '__invert__',
 '__le__',
 '__lshift__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__rrshift__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__trunc__',
 '__xor__',
 'bit_length',
 'conjugate',
 'denominator',
 'from_bytes',
 'imag',
 'numerator',
 'real',
 'to_bytes']

In [57]:
(1).bit_length()

1

This has implications for how assignment works in Python.

Let's create a trivial class:

In [58]:
class Thing: pass

and instantiate it:

In [59]:
x = Thing()
x

Here, `x` is simply a "label" for the object that we created when calling `Thing`. That object resides at the memory location that is identified when we print `x`. Notice that if we create another `Thing`, we create an new object, and give it a label. We know it is a new object because it has its own memory location.

In [60]:
y = Thing()
y

What if we assign `x` to `z`?

In [61]:
z = x
z

We see that the object labeled with `z` is the same as the object as that labeled with `x`. So, we say that `z` is a label (or name) with a *binding*  to the object created by `Thing`.

So, there are no "variables", in the sense of a container for values, in Python. There are only labels and bindings.

In [62]:
x.name = 'thing x'

In [63]:
z.name

'thing x'

This can get you into trouble. Consider the following (seemingly inoccuous) way of creating a dictionary of emtpy lists:

In [64]:
evil_dict = dict.fromkeys(column_names, [])
evil_dict

{'Group': [], 'Patient': [], 'Stool': [], 'Taxon': [], 'Tissue': []}

In [65]:
evil_dict['Tissue'].append(5)

In [66]:
evil_dict

{'Group': [5], 'Patient': [5], 'Stool': [5], 'Taxon': [5], 'Tissue': [5]}

Why did this happen?

## Exercise: Translate R to Python

Recode the secant search function from Bios 301 from R to Python.

```R
# %load http://git.io/-2DM8Q
f_test <- function(x) cos(x) - x

secant <- function(fun, x0, x1, tol=1e-9, max_iter=100) {
    # Keep track of number of interations
    iter <- 0
    # Evaluate function at initial points
    f1 <- fun(x1)
    f0 <- fun(x0)
    # Loop
    while((abs(x1-x0) > tol) && (iter<max_iter)) {
        # Calculate new value
        x_new <- x1 -  f1*(x1 - x0)/(f1 - f0)
        # Replace old value with current
        x0 <- x1
        x1 <- x_new
        f0 <- f1
        f1 <- fun(x1)
        # Increment counter
        iter <- iter + 1
    }

    if (abs(x1-x0) > tol) {
        cat("Algorithm failed to converge\n")
        return(NA)
    } 
    else {
        cat("Algorithm converged in", iter, "iterations\n")
        return(x1)
    }
}

secant(f_test, 2, 3)
```

In [67]:
# Write your code here

How about a modified secant functiont that uses a generator?

In [ ]:
# Write your code here

## References

* [Learn Python the Hard Way](http://learnpythonthehardway.org/book/)  
* [Learn X in Y Minutes (where X=Python)](http://learnxinyminutes.com/docs/python/)  
* [29 common beginner Python errors on one page](http://pythonforbiologists.com/index.php/29-common-beginner-python-errors-on-one-page/)
* [Understanding Python's Execution Model](http://www.jeffknupp.com/blog/2013/02/14/drastically-improve-your-python-understanding-pythons-execution-model/)